In [ ]:
import sys
from collections import deque

In [ ]:
def load_grid(fn):
    grid = {}
    
    with open(fn) as fh:
        for row, line in enumerate(fh):
            for col, c in enumerate(line.strip('\n')):
                grid[complex(col, row)] = c
                
    return grid

In [ ]:
def draw_grid(grid):
    max_x = max(int(p.real) for p in grid)
    max_y = max(int(p.imag) for p in grid)
    
    for row in range(max_y + 1):
        for col in range(max_x + 1):
            sys.stdout.write(grid[complex(col, row)])
        sys.stdout.write('\n')

In [ ]:
def find_portals(grid):
    portals = defaultdict(list)
    # portals will be k: tuples of (where will you arrive, where will you depart from)
    for p, c in grid.items():
        if c.isupper():
            for d in [1, 1j]:
                if p + d in grid and grid[p + d].isupper():
                    second_c = grid[p + d]
                    name = c + second_c
                    if p + 2 * d in grid and grid[p + 2 * d] == '.':
                        portals[name].append((p + 2 * d, p + d))
                    if p - d in grid and grid[p - d] == '.':
                        portals[name].append((p - d, p))
                    
    entrance = portals.pop('AA')[0][0]
    exit = portals.pop('ZZ')[0][0]
    
    connections = {}
    for name, endpoints in portals.items():
        arrive_1, depart_1 = endpoints[0]
        arrive_2, depart_2 = endpoints[1]
        
        connections[depart_1] = arrive_2
        connections[depart_2] = arrive_1
            
    return entrance, exit, connections

In [ ]:
def distances_from(start, grid, connections):
    distances = {start: 0}
    to_check = deque()

    to_check.append(start)

    while len(to_check) > 0:
        position = to_check.popleft()
        for direction in [1, -1, 1j, -1j]:
            next_position = position + direction
            next_position = connections.get(next_position, next_position)
            
            next_distance = distances[position] + 1
            next_char = grid[next_position]
            
            if next_char == '.' and next_position not in distances:
                distances[next_position] = next_distance
                to_check.append(next_position)
    
    return distances

In [ ]:
fn = 'day20.txt'

g = load_grid(fn)

entrance, exit, connections = find_portals(g)

ds = distances_from(entrance, g, connections)

print(ds[exit])

In [ ]:
import heapq

In [ ]:
def find_portals_part2(grid):
    portals = []
    # portals will be k: list of tuples (where will you arrive, inner or outer)
    for p, c in grid.items():
        if c.isupper():
            for d in [1, 1j]:
                if p + d in grid and grid[p + d].isupper():
                    second_c = grid[p + d]
                    name = c + second_c
                    if p + 2 * d in grid and grid[p + 2 * d] == '.':
                        if p - d in grid:
                            kind = 'inner'
                        else:
                            kind = 'outer'
                        portals.append((name, p + 2 * d, kind))
                    if p - d in grid and grid[p - d] == '.':
                        if p + 2 * d in grid:
                            kind = 'inner'
                        else:
                            kind = 'outer'
                        portals.append((name, p - d, kind))

    p_to_name = {p: (name, kind) for name, p, kind in portals}
    name_to_ps = defaultdict(dict)
    for name, p, kind in portals:
        name_to_ps[name][kind] = p
            
    return p_to_name, name_to_ps

In [ ]:
def make_connections(grid):
    p_to_name, name_to_p = find_portals_part2(grid)

    connections = {}

    for name in name_to_p:
        for side in ['outer', 'inner']:
            if side in name_to_p[name]:
                starting_p = name_to_p[name][side]
                connections[name, side] = [(p_to_name[p], d + 1) for p, d in distances_from(starting_p, grid, {}).items() if p in p_to_name and p != starting_p]
                
    return connections

In [ ]:
def dijkstra(connections, p):
    start = ('AA', 'outer', 0)
    
    distances = {start: 0}
    
    to_check = [(0, start)]

    while len(to_check) > 0:
        distance, (name, kind, level) = heapq.heappop(to_check)
        
        if (name, kind, level) == ('ZZ', 'inner', -1):
            distances[name, kind, level] = distance - 1
            break
        else:
            distances[name, kind, level] = distance
            
        for (next_name, connect_kind), distance_to in connections[name, kind]:
            if next_name == 'AA':
                continue
                
            if level == 0:
                if connect_kind == 'outer' and next_name not in ['AA', 'ZZ']:
                    continue
            elif level > 0:
                if connect_kind == 'outer' and next_name in ['AA', 'ZZ']:
                    continue
            
            next_distance = distance + distance_to
            
            if connect_kind == 'outer':
                next_level = level - 1
                next_kind = 'inner'                
            else:
                next_level = level + 1
                next_kind = 'outer'
                
            next_p = (next_name, next_kind, next_level)
                
            if next_p not in distances:
                heapq.heappush(to_check, (next_distance, next_p))
                
    return distances

In [ ]:
%%time

fn = 'day20.txt'

g = load_grid(fn)
cs = make_connections(g)

print(dijkstra(cs, p)[('ZZ', 'inner', -1)])